In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras as K

In [2]:
(x_train, y_train), (x_test, y_test) = tfds.as_numpy(tfds.load(
    'horses_or_humans',
    split=['train', 'test'],
    batch_size=-1,
    as_supervised=True,
))

x_train.shape, y_train.shape

((1027, 300, 300, 3), (1027,))

In [3]:
CLASSES = len(set(y_train.flatten().tolist()))
INPUT_SIZE = [300, 300, 3]

print(CLASSES)

2


In [4]:
def alexnet(input_size=INPUT_SIZE, activation='relu', dropout=False):
    model = K.Sequential([
        K.layers.Conv2D(96, (11, 11), (2, 2), input_shape=input_size),
        K.layers.BatchNormalization(),
        K.layers.Activation(activation),
        K.layers.MaxPool2D((3, 3), (2, 2)),
        
        K.layers.Conv2D(256, (5, 5), padding='same'),
        K.layers.BatchNormalization(),
        K.layers.Activation(activation),
        K.layers.MaxPool2D((3, 3), (2, 2)),

        K.layers.Conv2D(384, (3, 3), padding='same'),
        K.layers.BatchNormalization(),
        K.layers.Activation(activation),

        K.layers.Conv2D(384, (3, 3), padding='same'),
        K.layers.BatchNormalization(),
        K.layers.Activation(activation),

        K.layers.Conv2D(256, (3, 3), padding='same'),
        K.layers.BatchNormalization(),
        K.layers.Activation(activation),
        K.layers.MaxPool2D((3, 3), (2, 2)),

        K.layers.Flatten(),
        K.layers.Dense(9216, activation=activation),
    ])

    if dropout:
        model.add(K.layers.Dropout(0.5))

    model.add(K.layers.Dense(4096, activation=activation))

    if dropout:
        model.add(K.layers.Dropout(0.5))

    model.add(K.layers.Dense(4096, activation=activation))

    if dropout:
        model.add(K.layers.Dropout(0.5))

    model.add(K.layers.Dense(CLASSES, activation='softmax'))

    return model

In [5]:
model = alexnet()
model.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 145, 145, 96)      34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 145, 145, 96)      384       
_________________________________________________________________
activation (Activation)      (None, 145, 145, 96)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 72, 72, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 72, 72, 256)       1024      
_________________________________________________________________
activation_1 (Activation)    (None, 72, 72, 256)       0

In [ ]:
class max_acc_callback(K.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if (logs.get("accuracy") == 1.00 and logs.get("loss") < 0.03):
            print("\nReached 100% accuracy! Stopping training...")
            self.model.stop_training = True

tensorboard_callback = tf.keras.callbacks.TensorBoard("logs")

clf = model.fit(
    x_train,
    y_train,
    epochs=50,
    callbacks=[max_acc_callback(), tensorboard_callback]
)